In [27]:
import sys
sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

In [3]:
np.random.seed(1234)
tf.random.set_seed(1234)

In [1]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x, y, t, wind, leak, angle, u, v, c, k, w layers):
        
        X = np.concatenate([x, y, t, wind, leak, angle], 1)
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        self.wind = X[:,3:4]
        self.leak = X[:,4:5]
        self.angle = X[:,5:6]
        
        self.u = u
        self.v = v
        self.c = c
        self.c = k
        self.c = w
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.rho = tf.Variable([0.0], dtype=tf.float32)
        #self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.wind_tf = tf.placeholder(tf.float32, shape=[None, self.wind.shape[1]])
        self.leak_tf = tf.placeholder(tf.float32, shape=[None, self.leak.shape[1]])
        self.angle_tf = tf.placeholder(tf.float32, shape=[None, self.angle.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
        self.v_tf = tf.placeholder(tf.float32, shape=[None, self.v.shape[1]])
        self.c_tf = tf.placeholder(tf.float32, shape=[None, self.c.shape[1]])
        self.k_tf = tf.placeholder(tf.float32, shape=[None, self.c.shape[1]])
        self.w_tf = tf.placeholder(tf.float32, shape=[None, self.c.shape[1]])
        
        self.u_pred, self.v_pred, self.c_pred, self.f_continuity, self.f_convection = self.net_NS(self.x_tf, self.y_tf, self.t_tf, self.wind_tf, self.leak_tf, self.angle_tf)
        
        # Modify the loss function to consider the importance of c
        self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_sum(tf.square(self.v_tf - self.v_pred)) + \
                    1 * tf.reduce_sum(tf.square(self.c_tf - self.c_pred)) + \
                    tf.reduce_sum(tf.square(self.f_continuity)) + \
                    tf.reduce_sum(tf.square(self.f_convection))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

In [10]:
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    def net_NS(self, x, y, t, wind, leak, angle):
        
        # Concatenating all input variables
        input_data = tf.concat([x, y, t, wind, leak, angle], 1)
    
        # Assuming the neural network is modified to output u, v, and c, k ,w
        u_v_c_k_w = self.neural_net(input_data, self.weights, self.biases)
    
        u = u_v_c_k_w[:,0:1]
        v = u_v_c_k_w[:,1:2]
        c = u_v_c_k_w[:,2:3]
        k = u_v_c_k_w[:,3:4]
        w = u_v_c_k_w[:,4:5]
    
        # Calculating the required derivatives
        u_x = tf.gradients(u, x)[0]
        v_y = tf.gradients(v, y)[0]
        c_x = tf.gradients(c, x)[0]
        c_y = tf.gradients(c, y)[0]
        c_xx = tf.gradients(c_x, x)[0]
        c_yy = tf.gradients(c_y, y)[0]
        
        f_continuity = u_x + v_y
        
        f_convection = self.compute_f_convection(u, v, c, x, y, t, wind, leak, angle)
        
        return u, v, c, f_continuity, f_convection # we can include k , w 

    def compute_f_convection(self, u, v, c, u_x, v_y, c_x, c_y, c_t, c_xx, c_yy, k, w):
        tranisent_term = c_t
        convective_term = c * (u_x + v_y) + u*c_x + v*c_y 
        diffusive_term = (1.7e-05 + (2.08e-5*k / w) / 0.803) * (c_xx, c_yy)
        return transient_term + convective_term + diffusive_term

In [11]:
    def callback(self, loss):
        print('Loss: %.3e, l1: %.3f, l2: %.5f' % (loss, loss, loss))
      
    def train(self, nIter): 
        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.t_tf: self.t,
                   self.wind_tf: self.wind, self.leak_tf: self.leak, self.angle_tf: self.angle,
                   self.u_tf: self.u, self.v_tf: self.v, self.c_tf: self.c}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                #lambda_1_value = self.sess.run(self.lambda_1)
                #lambda_2_value = self.sess.run(self.lambda_2)
                print('It: %d, Loss: %.3e, Time: %.2f' % 
                      (it, loss_value, elapsed))
                start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
            
    
    def predict(self, x_star, y_star, t_star, wind_star, leak_star, angle_star):
        tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star,
                     self.wind_tf: wind_star, self.leak_tf: leak_star, self.angle_tf: angle_star}
        u_star = self.sess.run(self.u_pred, tf_dict)
        v_star = self.sess.run(self.v_pred, tf_dict)
        c_star = self.sess.run(self.p_pred, tf_dict)
        return u_star, v_star, c_star


In [ ]:
if __name__ == "__main__": 
      
    N_train = 5000
    
    layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
    
    # Load Data (replace with my data loader)
    # data = scipy.io.loadmat('../Data/cylinder_nektar_wake.mat')
           
    U_star = data['U_star'] # N x 2 x T
    P_star = data['p_star'] # N x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    PP = P_star # N x T
    
    x = XX.flatten()[:,None] # NT x 1
    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    
    u = UU.flatten()[:,None] # NT x 1
    v = VV.flatten()[:,None] # NT x 1
    p = PP.flatten()[:,None] # NT x 1
    
    ######################################################################
    ######################## Noiseles Data ###############################
    ######################################################################
    # Training Data    
    idx = np.random.choice(N*T, N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]

    # Training
    model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
    model.train(200000)
    
    # Test Data
    snap = np.array([100])
    x_star = X_star[:,0:1]
    y_star = X_star[:,1:2]
    t_star = TT[:,snap]
    
    u_star = U_star[:,0,snap]
    v_star = U_star[:,1,snap]
    p_star = P_star[:,snap]
    
    # Prediction
    u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star)
    lambda_1_value = model.sess.run(model.lambda_1)
    lambda_2_value = model.sess.run(model.lambda_2)
    
    # Error
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
    error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

    error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
    error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    
    print('Error u: %e' % (error_u))    
    print('Error v: %e' % (error_v))    
    print('Error p: %e' % (error_p))    
    print('Error l1: %.5f%%' % (error_lambda_1))                             
    print('Error l2: %.5f%%' % (error_lambda_2))                  
    
    # Plot Results
#    plot_solution(X_star, u_pred, 1)
#    plot_solution(X_star, v_pred, 2)
#    plot_solution(X_star, p_pred, 3)    
#    plot_solution(X_star, p_star, 4)
#    plot_solution(X_star, p_star - p_pred, 5)
    
    # Predict for plotting
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    UU_star = griddata(X_star, u_pred.flatten(), (X, Y), method='cubic')
    VV_star = griddata(X_star, v_pred.flatten(), (X, Y), method='cubic')
    PP_star = griddata(X_star, p_pred.flatten(), (X, Y), method='cubic')
    P_exact = griddata(X_star, p_star.flatten(), (X, Y), method='cubic')


In [ ]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)